In [1]:
import numpy as np
import pandas as pd
import polars as pl
import sys
import re
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px


pd.set_option('display.max_columns',None)
import psycopg2


#to scale the data using z-score 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Algorithms to use
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

#Metrics to evaluate the model
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve

import warnings
warnings.filterwarnings("ignore")

#importing PCA and TSNE
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [30]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL4_processed_vcf.xlsx')
df

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,IMPACT,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN


In [31]:
df['consequence'] = df['Consequence'].str.split(',').str[0]
df

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,IMPACT,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,consequence
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN,5 prime UTR variant
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN,intron variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN,synonymous variant
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN,synonymous variant
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN,intron variant
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN,intron variant


In [32]:
df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')
df_1

,consequence,Consequence_score
0,transcript ablation,10/10
1,splice acceptor variant,8/10
2,splice donor variant,8/10
3,stop gained,10/10
4,frameshift variant,10/10
5,stop lost,9/10
6,start lost,9/10
7,transcript amplification,8/10
8,inframe insertion,6/10
9,inframe deletion,6/10


In [33]:
merged_1 = pd.merge(df, df_1, on='consequence', how='left', sort=False)
merged_1

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,IMPACT,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,consequence,Consequence_score
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant,2/10
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN,5 prime UTR variant,3/10
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant,2/10
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant,2/10
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN,intron variant,2/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN,synonymous variant,3/10
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN,synonymous variant,3/10
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN,intron variant,2/10
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN,intron variant,2/10


In [34]:
merged_1[merged_1['ID'] == 'rs112703963']

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,IMPACT,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,consequence,Consequence_score
16,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,NaN,NaN,NaN,NaN,1.0,NaN,upstream gene variant,2/10
17,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,-1.0,NaN,upstream gene variant,2/10
18,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,NaN,NaN,NaN,NaN,1.0,NaN,upstream gene variant,2/10
19,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,retained intron,NaN,NaN,NaN,NaN,-1.0,NaN,upstream gene variant,2/10
20,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000616016.5,c.102A>G,ENSP00000478421.2,p.Pro34%3D,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,1/14,NaN,P34,ccA/ccG,1.0,NaN,synonymous variant,3/10
21,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,NaN,NaN,NaN,NaN,1.0,NaN,upstream gene variant,2/10
22,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,NaN,NaN,NaN,NaN,1.0,NaN,upstream gene variant,2/10
23,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,NaN,NaN,NaN,NaN,1.0,NaN,upstream gene variant,2/10
24,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000618323.5,c.102A>G,ENSP00000480678.2,p.Pro34%3D,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,1/14,NaN,P34,ccA/ccG,1.0,NaN,synonymous variant,3/10
25,"LOC107985728,SAMD11",rs112703963,chr1,924533,A,G,Homozygous,upstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,1/1,200,35,35,0,35,100%,8.913700e-21,0,0,22,13,NaN,NaN,protein coding,NaN,NaN,NaN,NaN,1.0,NaN,upstream gene variant,2/10


In [35]:
df_2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/IMPACT.xlsx')
df_2

,IMPACT,IMPACT_score
0,HIGH,10.0
1,MODERATE,5.0
2,LOW,2.5
3,MODIFIER,1.5


In [36]:
merged_2 = pd.merge(merged_1, df_2, on = 'IMPACT', how='left', sort=False)
merged_2

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,IMPACT,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,consequence,Consequence_score,IMPACT_score
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant,2/10,1.5
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN,5 prime UTR variant,3/10,1.5
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant,2/10,1.5
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,MODIFIER,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,downstream gene variant,2/10,1.5
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN,intron variant,2/10,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN,synonymous variant,3/10,2.5
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,LOW,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN,synonymous variant,3/10,2.5
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN,intron variant,2/10,1.5
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,MODIFIER,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN,intron variant,2/10,1.5


In [37]:
merged_2 = merged_2[['Gene Name', 'ID', 'CHROM', 'POS', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]
merged_2

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,3/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN


In [47]:
merged_2.ID.value_counts()

.             2424
rs12624152     145
rs848291       144
rs2589516      123
rs183956       112
              ... 
rs1197683        1
rs2275443        1
rs61746215       1
rs12411843       1
rs2900373        1
Name: ID, Length: 35804, dtype: int64

In [48]:
merged_2[merged_2['ID'] == 'rs12624152']

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
182712,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,intron variant,2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000233741.9,c.692-27A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,protein coding,NaN,8/13,NaN,NaN,-1.0,NaN
182713,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,NaN,None,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,protein coding,NaN,NaN,NaN,NaN,1.0,NaN
182714,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,intron variant,2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000402135.8,c.707-27A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,protein coding,NaN,8/13,NaN,NaN,-1.0,NaN
182715,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,intron variant,2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000403295.8,c.692-470A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,protein coding,NaN,8/12,NaN,NaN,-1.0,NaN
182716,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,"intron variant,NMD transcript variant",2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000403676.6,c.*255-27A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,nonsense mediated decay,NaN,4/9,NaN,NaN,-1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182852,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,intron variant,2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000696635.1,c.692-51A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,protein coding,NaN,8/13,NaN,NaN,-1.0,NaN
182853,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,"intron variant,non coding transcript variant",2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000696636.1,n.1477-27A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,retained intron,NaN,1/2,NaN,NaN,-1.0,NaN
182854,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,non coding transcript exon variant,2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000696637.1,n.7298A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,retained intron,3/6,NaN,NaN,NaN,-1.0,NaN
182855,"VRK2,FANCL",rs12624152,chr2,58163544,T,G,Heterozygous,"intron variant,non coding transcript variant",2/10,MODIFIER,1.5,not provided,benign,"criteria provided, single submitter",1277221.0,ENST00000696638.1,n.707-27A>C,NaN,None,0/1,50,21,21,8,13,61.9%,0.000008,4,4,9,4,NaN,NaN,retained intron,NaN,4/9,NaN,NaN,-1.0,NaN


In [38]:
merged_2.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL4_depth_vcf_processed.xlsx', index=False)

In [28]:
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL4_depth_vcf_processed.xlsx')
df['POS'] = df['POS'].astype('int64')
df

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,3/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN


In [29]:
df_3 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/Cardiac_pos.xlsx')
df_3

,POS,Literature
0,104858586,Yes
1,104826974,Yes
2,104824472,Yes
3,104858554,Yes
4,104794495,Yes
...,...,...
1861,31592980,Yes
1862,31595223,Yes
1863,31593004,Yes
1864,31598691,Yes


In [30]:
merged_3 = pd.merge(df, df_3, on = 'POS', how = 'left', sort=False)
merged_3

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,Literature
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,3/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN,NaN
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN,NaN
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,NaN
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN,NaN
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN,NaN
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN,NaN
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN,NaN


In [31]:
merged_3['Literature'] = merged_3['Literature'].fillna('No')
merged_3

,Gene Name,ID,CHROM,POS,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,Literature
0,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,No
1,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,5 prime UTR variant,3/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN,No
2,NaN,rs1473003496,chr1,65529,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN,No
3,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,No
4,NaN,rs1416222198,chr1,65591,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN,No
316575,NaN,.,chrY,23190581,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN,No
316576,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN,No
316577,NaN,.,chrY,23191171,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN,No


In [32]:
merged_3.Literature.value_counts()

No     316485
Yes        94
Name: Literature, dtype: int64

In [33]:
merged_3[merged_3['Literature'] == 'Yes'].ID.value_counts()

rs1071646     39
rs1805127      8
rs1799983      5
rs5882         4
rs1800777      4
rs6771157      4
rs59468016     4
rs7374804      4
rs2230806      4
rs7521023      3
rs6599242      3
rs78425180     3
rs7630989      3
rs7617919      3
rs74717885     3
Name: ID, dtype: int64

In [34]:
merged_3 = merged_3[['Gene Name', 'ID', 'CHROM', 'POS', 'Literature', 'REF', 'ALT', 'Zygosity',
       'Consequence','Consequence_score', 'IMPACT', 'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG',
       'ClinVar_CLNREVSTAT', 'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp',
       'HGVSp (Transcript)', 'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ',
       'PVAL', 'RDF', 'RDR', 'ADF', 'ADR', 'SIFT', 'PolyPhen', 'BIOTYPE',
       'EXON', 'INTRON', 'Protein Position and Amino Acid', 'Codons', 'STRAND',
       'PUBMED']]
merged_3

,Gene Name,ID,CHROM,POS,Literature,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
0,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,5 prime UTR variant,3/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.-36C>T,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN
2,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,23,26,26,19,7,26.92%,0.004917,18,1,5,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,rs1416222198,chr1,65591,No,C,T,Heterozygous,downstream gene variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,NaN,None,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
4,NaN,rs1416222198,chr1,65591,No,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000641515.2,c.9+18C>T,NaN,None,0/1,22,28,28,21,7,25%,0.005105,17,4,6,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.351A>G,ENSP00000400716.3,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN
316575,NaN,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,3/10,LOW,2.5,NaN,NaN,NaN,NaN,ENST00000540248.5,c.351A>G,ENSP00000444407.1,p.Gln117%3D,0/1,48,73,73,58,15,20.55%,0.000014,38,20,9,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN
316576,NaN,.,chrY,23191171,No,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000405239.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN
316577,NaN,.,chrY,23191171,No,C,T,Heterozygous,intron variant,2/10,MODIFIER,1.5,NaN,NaN,NaN,NaN,ENST00000426000.6,c.243-46G>A,NaN,None,0/1,26,33,33,25,8,24.24%,0.002417,17,8,7,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN


In [35]:
merged_3.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL4_depth_vcf_processed.xlsx', index= False)

In [1]:
import pandas as pd

# Create sample data
data1 = {'Gene': ['OR4F29', 'LOC107985728,SAMD11', 'ABCD1,XYZ']}
df1 = pd.DataFrame(data1)

data2 = {'Gene': ['ABCD1', 'XYZ', 'EFGH', 'IJKL']}
df2 = pd.DataFrame(data2)

In [3]:
df2

,Gene
0,ABCD1
1,XYZ
2,EFGH
3,IJKL


In [37]:
df1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL4_depth_vcf_processed.xlsx')
df1

,Gene Name,ID,CHROM,POS,Literature,REF,ALT,Zygosity,Consequence,Consequence_score,...,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
0,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,2/10,...,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,5 prime UTR variant,3/10,...,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN
2,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,2/10,...,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,rs1416222198,chr1,65591,No,C,T,Heterozygous,downstream gene variant,2/10,...,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
4,NaN,rs1416222198,chr1,65591,No,C,T,Heterozygous,intron variant,2/10,...,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,3/10,...,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN
316575,NaN,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,3/10,...,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN
316576,NaN,.,chrY,23191171,No,C,T,Heterozygous,intron variant,2/10,...,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN
316577,NaN,.,chrY,23191171,No,C,T,Heterozygous,intron variant,2/10,...,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN


In [38]:
df2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/cardiac genes.xlsx')
df2

,Gene Name
0,LDLR
1,LDLR-AS1
2,NOS3
3,PRKAG2
4,LRP6
...,...
75,TRDN
76,LDLRAP1
77,PCSK9
78,APOB


In [39]:
df1['Gene_Match'] = 'No'

# Iterate through each gene in df1
for genes in df1['Gene Name']:
    if isinstance(genes, str):  # Check if the gene value is a non-null string
        gene_list = genes.split(',')  # Split the genes by comma to create a list
        match = any(gene in df2['Gene Name'].values for gene in gene_list)  # Check if any gene in the list exists in df2
        if match:
            df1.loc[df1['Gene Name'] == genes, 'Gene_Match'] = 'Yes'
            
df1

,Gene Name,ID,CHROM,POS,Literature,REF,ALT,Zygosity,Consequence,Consequence_score,...,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,Gene_Match
0,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,2/10,...,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,No
1,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,5 prime UTR variant,3/10,...,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN,No
2,NaN,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,2/10,...,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN,No
3,NaN,rs1416222198,chr1,65591,No,C,T,Heterozygous,downstream gene variant,2/10,...,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN,No
4,NaN,rs1416222198,chr1,65591,No,C,T,Heterozygous,intron variant,2/10,...,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,3/10,...,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN,No
316575,NaN,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,3/10,...,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN,No
316576,NaN,.,chrY,23191171,No,C,T,Heterozygous,intron variant,2/10,...,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN,No
316577,NaN,.,chrY,23191171,No,C,T,Heterozygous,intron variant,2/10,...,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN,No


In [40]:
df1[df1['Gene_Match'] == 'Yes']

,Gene Name,ID,CHROM,POS,Literature,REF,ALT,Zygosity,Consequence,Consequence_score,...,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED,Gene_Match
7713,LDLRAP1,rs6687605,chr1,25563141,No,T,C,Heterozygous,missense variant,7/10,...,deleterious(0.04),benign(0.037),protein coding,6/9,NaN,S202P,Tcc/Ccc,1.0,"31227787,24906453,31397093,26129832,24067191",Yes
7714,LDLRAP1,rs6687605,chr1,25563141,No,T,C,Heterozygous,upstream gene variant,2/10,...,NaN,NaN,protein coding CDS not defined,NaN,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191",Yes
7715,LDLRAP1,rs6687605,chr1,25563141,No,T,C,Heterozygous,upstream gene variant,2/10,...,NaN,NaN,protein coding CDS not defined,NaN,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191",Yes
7716,LDLRAP1,rs6687605,chr1,25563141,No,T,C,Heterozygous,non coding transcript exon variant,2/10,...,NaN,NaN,protein coding CDS not defined,1/4,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191",Yes
7717,LDLRAP1,rs6687605,chr1,25563141,No,T,C,Heterozygous,non coding transcript exon variant,2/10,...,NaN,NaN,protein coding CDS not defined,5/7,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191",Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304958,ABCA1,rs2230806,chr9,104858586,Yes,C,T,Homozygous,missense variant,7/10,...,tolerated low confidence(1),benign(0),protein coding,7/50,NaN,R219K,aGg/aAg,-1.0,"23021345,23497168,28690685,20185793,30584432,2...",Yes
304959,ABCA1,rs2230805,chr9,104861748,No,C,T,Homozygous,synonymous variant,3/10,...,NaN,NaN,protein coding,5/5,NaN,L98,ctG/ctA,-1.0,"20167577,22675311,29156910,19606474,22369239,1...",Yes
304960,ABCA1,rs2230805,chr9,104861748,No,C,T,Homozygous,synonymous variant,3/10,...,NaN,NaN,protein coding,6/50,NaN,L158,ctG/ctA,-1.0,"20167577,22675311,29156910,19606474,22369239,1...",Yes
304961,ABCA1,rs2230805,chr9,104861748,No,C,T,Homozygous,synonymous variant,3/10,...,NaN,NaN,protein coding,6/8,NaN,L158,ctG/ctA,-1.0,"20167577,22675311,29156910,19606474,22369239,1...",Yes


In [41]:
df1.columns

Index(['Gene Name', 'ID', 'CHROM', 'POS', 'Literature', 'REF', 'ALT',
       'Zygosity', 'Consequence', 'Consequence_score', 'IMPACT',
       'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT',
       'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)',
       'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR',
       'ADF', 'ADR', 'SIFT', 'PolyPhen', 'BIOTYPE', 'EXON', 'INTRON',
       'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED',
       'Gene_Match'],
      dtype='object')

In [42]:
df1 = df1[['Gene Name', 'Gene_Match', 'ID', 'CHROM', 'POS', 'Literature', 'REF', 'ALT',
       'Zygosity', 'Consequence', 'Consequence_score', 'IMPACT',
       'IMPACT_score', 'ClinVar_CLNDN', 'CLIN_SIG', 'ClinVar_CLNREVSTAT',
       'ClinVar', 'HGVSc', 'HGVSc (Transcript)', 'HGVSp', 'HGVSp (Transcript)',
       'GT', 'GQ', 'SDP', 'DP', 'RD', 'AD', 'FREQ', 'PVAL', 'RDF', 'RDR',
       'ADF', 'ADR', 'SIFT', 'PolyPhen', 'BIOTYPE', 'EXON', 'INTRON',
       'Protein Position and Amino Acid', 'Codons', 'STRAND', 'PUBMED']]
df1

,Gene Name,Gene_Match,ID,CHROM,POS,Literature,REF,ALT,Zygosity,Consequence,...,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
0,NaN,No,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,...,2,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
1,NaN,No,rs1473003496,chr1,65529,No,C,T,Heterozygous,5 prime UTR variant,...,2,NaN,NaN,protein coding,2/3,NaN,NaN,NaN,1.0,NaN
2,NaN,No,rs1473003496,chr1,65529,No,C,T,Heterozygous,downstream gene variant,...,2,NaN,NaN,lncRNA,NaN,NaN,NaN,NaN,1.0,NaN
3,NaN,No,rs1416222198,chr1,65591,No,C,T,Heterozygous,downstream gene variant,...,1,NaN,NaN,transcribed unprocessed pseudogene,NaN,NaN,NaN,NaN,1.0,NaN
4,NaN,No,rs1416222198,chr1,65591,No,C,T,Heterozygous,intron variant,...,1,NaN,NaN,protein coding,NaN,2/2,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316574,NaN,No,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,...,6,NaN,NaN,protein coding,5/16,NaN,Q117,caA/caG,-1.0,NaN
316575,NaN,No,.,chrY,23190581,No,T,C,Heterozygous,synonymous variant,...,6,NaN,NaN,protein coding,5/17,NaN,Q117,caA/caG,-1.0,NaN
316576,NaN,No,.,chrY,23191171,No,C,T,Heterozygous,intron variant,...,1,NaN,NaN,protein coding,NaN,3/27,NaN,NaN,-1.0,NaN
316577,NaN,No,.,chrY,23191171,No,C,T,Heterozygous,intron variant,...,1,NaN,NaN,protein coding,NaN,3/15,NaN,NaN,-1.0,NaN


In [47]:
df1[df1['Gene_Match'] == 'Yes']

,Gene Name,Gene_Match,ID,CHROM,POS,Literature,REF,ALT,Zygosity,Consequence,Consequence_score,IMPACT,IMPACT_score,ClinVar_CLNDN,CLIN_SIG,ClinVar_CLNREVSTAT,ClinVar,HGVSc,HGVSc (Transcript),HGVSp,HGVSp (Transcript),GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,SIFT,PolyPhen,BIOTYPE,EXON,INTRON,Protein Position and Amino Acid,Codons,STRAND,PUBMED
7713,LDLRAP1,Yes,rs6687605,chr1,25563141,No,T,C,Heterozygous,missense variant,7/10,MODERATE,5.0,"Hypercholesterolemia, familial, 4,Cardiovascul...",benign,"criteria provided, multiple submitters, no con...",296982.0,ENST00000374338.5,c.604T>C,ENSP00000363458.4,p.Ser202Pro,0/1,25,15,15,8,7,46.67%,3.160900e-03,6,2,5,2,deleterious(0.04),benign(0.037),protein coding,6/9,NaN,S202P,Tcc/Ccc,1.0,"31227787,24906453,31397093,26129832,24067191"
7714,LDLRAP1,Yes,rs6687605,chr1,25563141,No,T,C,Heterozygous,upstream gene variant,2/10,MODIFIER,1.5,"Hypercholesterolemia, familial, 4,Cardiovascul...",benign,"criteria provided, multiple submitters, no con...",296982.0,NaN,None,NaN,None,0/1,25,15,15,8,7,46.67%,3.160900e-03,6,2,5,2,NaN,NaN,protein coding CDS not defined,NaN,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191"
7715,LDLRAP1,Yes,rs6687605,chr1,25563141,No,T,C,Heterozygous,upstream gene variant,2/10,MODIFIER,1.5,"Hypercholesterolemia, familial, 4,Cardiovascul...",benign,"criteria provided, multiple submitters, no con...",296982.0,NaN,None,NaN,None,0/1,25,15,15,8,7,46.67%,3.160900e-03,6,2,5,2,NaN,NaN,protein coding CDS not defined,NaN,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191"
7716,LDLRAP1,Yes,rs6687605,chr1,25563141,No,T,C,Heterozygous,non coding transcript exon variant,2/10,MODIFIER,1.5,"Hypercholesterolemia, familial, 4,Cardiovascul...",benign,"criteria provided, multiple submitters, no con...",296982.0,ENST00000484476.5,n.326T>C,NaN,None,0/1,25,15,15,8,7,46.67%,3.160900e-03,6,2,5,2,NaN,NaN,protein coding CDS not defined,1/4,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191"
7717,LDLRAP1,Yes,rs6687605,chr1,25563141,No,T,C,Heterozygous,non coding transcript exon variant,2/10,MODIFIER,1.5,"Hypercholesterolemia, familial, 4,Cardiovascul...",benign,"criteria provided, multiple submitters, no con...",296982.0,ENST00000488127.1,n.1074T>C,NaN,None,0/1,25,15,15,8,7,46.67%,3.160900e-03,6,2,5,2,NaN,NaN,protein coding CDS not defined,5/7,NaN,NaN,NaN,1.0,"31227787,24906453,31397093,26129832,24067191"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304958,ABCA1,Yes,rs2230806,chr9,104858586,Yes,C,T,Homozygous,missense variant,7/10,MODERATE,5.0,"Cardiovascular phenotype,Hypoalphalipoproteine...",benign,"criteria provided, multiple submitters, no con...",9506.0,ENST00000678995.1,c.656G>A,ENSP00000504612.1,p.Arg219Lys,1/1,206,36,36,0,36,100%,2.259800e-21,0,0,24,12,tolerated low confidence(1),benign(0),protein coding,7/50,NaN,R219K,aGg/aAg,-1.0,"23021345,23497168,28690685,20185793,30584432,2..."
304959,ABCA1,Yes,rs2230805,chr9,104861748,No,C,T,Homozygous,synonymous variant,3/10,LOW,2.5,"Cardiovascular phenotype,Hypoalphalipoproteine...",benign,"criteria provided, multiple submitters, no con...",364460.0,ENST00000374733.1,c.294G>A,ENSP00000363865.1,p.Leu98%3D,1/1,64,12,12,0,12,100%,3.698000e-07,0,0,10,2,NaN,NaN,protein coding,5/5,NaN,L98,ctG/ctA,-1.0,"20167577,22675311,29156910,19606474,22369239,1..."
304960,ABCA1,Yes,rs2230805,chr9,104861748,No,C,T,Homozygous,synonymous variant,3/10,LOW,2.5,"Cardiovascular phenotype,Hypoalphalipoproteine...",benign,"criteria provided, multiple submitters, no con...",364460.0,ENST00000374736.8,c.474G>A,ENSP00000363868.3,p.Leu158%3D,1/1,64,12,12,0,12,100%,3.698000e-07,0,0,10,2,NaN,NaN,protein coding,6/50,NaN,L158,ctG/ctA,-1.0,"20167577,22675311,29156910,19606474,22369239,1..."
304961,ABCA1,Yes,rs2230805,chr9,104861748,No,C,T,Homozygous,synonymous variant,3/10,LOW,2.5,"Cardiovascular phenotype,Hypoalphalipoproteine...

In [34]:
df1 = df1.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Processed_vcf_files/KHCDPRGPTTL4_Gene_depth_vcf_processed.xlsx', index=False)

## InDel files Analysis

In [74]:
import pandas as pd
df = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHGLBS535/KHGLBS535_indel_clinical_significance.xlsx')
df

,allele,zygocity,gene_name,rsid,consequence,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious
0,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No
1,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
2,"['ACCTT', A]",Heterozygous,ADAMTS17,rs10549565,3_prime_UTR_variant,Benign,"Weill-Marchesani_4_syndrome', '_recessive|not_...","criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,No
3,"['CAT', C]",Homozygous,PTGES3,rs10579382,"initiatior_codon_variant', 'SO:0001589",Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No
4,"['CTGTG', C]",Heterozygous,ELN,rs10579871,intron_variant,Conflicting_interpretations_of_pathogenicity,"Supravalvar_aortic_stenosis|Cutis_laxa', '_aut...","criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Benign,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
577,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Uncertain_significance,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
578,"['CT', C]",Heterozygous,FANCD2,rs797045572,intron_variant,Uncertain_significance,not_specified,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes
579,"['C', CA]",Heterozygous,HSD17B3,rs8190483,intron_variant,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes


In [75]:
df2 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Vitiligo_genes.xlsx')
df2

,Gene Name
0,HLADR4
1,HLADR7
2,HLADQ7
3,HLADR1
4,HLAB13
5,HLADQW3
6,HLACW6
7,HLAA30
8,AIS1
9,FOXD3


In [76]:
df = df.rename({'gene_name':'Gene Name'}, axis = 1)
df

,allele,zygocity,Gene Name,rsid,consequence,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious
0,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No
1,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
2,"['ACCTT', A]",Heterozygous,ADAMTS17,rs10549565,3_prime_UTR_variant,Benign,"Weill-Marchesani_4_syndrome', '_recessive|not_...","criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,No
3,"['CAT', C]",Homozygous,PTGES3,rs10579382,"initiatior_codon_variant', 'SO:0001589",Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No
4,"['CTGTG', C]",Heterozygous,ELN,rs10579871,intron_variant,Conflicting_interpretations_of_pathogenicity,"Supravalvar_aortic_stenosis|Cutis_laxa', '_aut...","criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Benign,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
577,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Uncertain_significance,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
578,"['CT', C]",Heterozygous,FANCD2,rs797045572,intron_variant,Uncertain_significance,not_specified,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes
579,"['C', CA]",Heterozygous,HSD17B3,rs8190483,intron_variant,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes


In [77]:
df['Gene_Match'] = 'No'

# Iterate through each gene in df1
for genes in df['Gene Name']:
    if isinstance(genes, str):  # Check if the gene value is a non-null string
        gene_list = genes.split(',')  # Split the genes by comma to create a list
        match = any(gene in df2['Gene Name'].values for gene in gene_list)  # Check if any gene in the list exists in df2
        if match:
            df.loc[df['Gene Name'] == genes, 'Gene_Match'] = 'Yes'
            
df

,allele,zygocity,Gene Name,rsid,consequence,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious,Gene_Match
0,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No,No
1,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No
2,"['ACCTT', A]",Heterozygous,ADAMTS17,rs10549565,3_prime_UTR_variant,Benign,"Weill-Marchesani_4_syndrome', '_recessive|not_...","criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,No,No
3,"['CAT', C]",Homozygous,PTGES3,rs10579382,"initiatior_codon_variant', 'SO:0001589",Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No,No
4,"['CTGTG', C]",Heterozygous,ELN,rs10579871,intron_variant,Conflicting_interpretations_of_pathogenicity,"Supravalvar_aortic_stenosis|Cutis_laxa', '_aut...","criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Benign,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No
577,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Uncertain_significance,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No
578,"['CT', C]",Heterozygous,FANCD2,rs797045572,intron_variant,Uncertain_significance,not_specified,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes,No
579,"['C', CA]",Heterozygous,HSD17B3,rs8190483,intron_variant,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes,No


In [78]:
df.Gene_Match.value_counts()

No    581
Name: Gene_Match, dtype: int64

In [80]:
df.consequence.fillna("NA", inplace=True)
df

,allele,zygocity,Gene Name,rsid,consequence,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious,Gene_Match
0,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No,No
1,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No
2,"['ACCTT', A]",Heterozygous,ADAMTS17,rs10549565,3_prime_UTR_variant,Benign,"Weill-Marchesani_4_syndrome', '_recessive|not_...","criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,No,No
3,"['CAT', C]",Homozygous,PTGES3,rs10579382,"initiatior_codon_variant', 'SO:0001589",Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No,No
4,"['CTGTG', C]",Heterozygous,ELN,rs10579871,intron_variant,Conflicting_interpretations_of_pathogenicity,"Supravalvar_aortic_stenosis|Cutis_laxa', '_aut...","criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Benign,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No
577,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Uncertain_significance,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No
578,"['CT', C]",Heterozygous,FANCD2,rs797045572,intron_variant,Uncertain_significance,not_specified,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes,No
579,"['C', CA]",Heterozygous,HSD17B3,rs8190483,intron_variant,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes,No


In [81]:
df.consequence.isnull().sum()

0

In [82]:
df['Consequence'] = df['consequence'].astype(str).apply(lambda x: x.replace('&', ',').replace('_', ' ').replace("'", '').replace("-", ' '))
df['Consequence'] = df['Consequence'].str.split(',').str[0]
df

,allele,zygocity,Gene Name,rsid,consequence,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious,Gene_Match,Consequence
0,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No,No,5 prime UTR variant
1,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No,5 prime UTR variant
2,"['ACCTT', A]",Heterozygous,ADAMTS17,rs10549565,3_prime_UTR_variant,Benign,"Weill-Marchesani_4_syndrome', '_recessive|not_...","criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,No,No,3 prime UTR variant
3,"['CAT', C]",Homozygous,PTGES3,rs10579382,"initiatior_codon_variant', 'SO:0001589",Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No,No,initiatior codon variant
4,"['CTGTG', C]",Heterozygous,ELN,rs10579871,intron_variant,Conflicting_interpretations_of_pathogenicity,"Supravalvar_aortic_stenosis|Cutis_laxa', '_aut...","criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes,No,intron variant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Benign,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No,3 prime UTR variant
577,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Uncertain_significance,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No,3 prime UTR variant
578,"['CT', C]",Heterozygous,FANCD2,rs797045572,intron_variant,Uncertain_significance,not_specified,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes,No,intron variant
579,"['C', CA]",Heterozygous,HSD17B3,rs8190483,intron_variant,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes,No,intron variant


In [83]:
df_1 = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Madhu_folder_04_07_2023/kidney_health_final.vcf/consequence.xlsx')
df_1 = df_1.rename({'consequence':'Consequence'}, axis =1)
df_1

,Consequence,Consequence_score
0,transcript ablation,10/10
1,splice acceptor variant,8/10
2,splice donor variant,8/10
3,stop gained,10/10
4,frameshift variant,10/10
5,stop lost,9/10
6,start lost,9/10
7,transcript amplification,8/10
8,inframe insertion,6/10
9,inframe deletion,6/10


In [84]:
merged_1 = pd.merge(df, df_1, on='Consequence', how='left', sort=False)
merged_1

,allele,zygocity,Gene Name,rsid,consequence,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious,Gene_Match,Consequence,Consequence_score
0,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No,No,5 prime UTR variant,3/10
1,"['AAGAGAG', A]",Heterozygous,TRPS1,rs10546472,5_prime_UTR_variant,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No,5 prime UTR variant,3/10
2,"['ACCTT', A]",Heterozygous,ADAMTS17,rs10549565,3_prime_UTR_variant,Benign,"Weill-Marchesani_4_syndrome', '_recessive|not_...","criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,No,No,3 prime UTR variant,3/10
3,"['CAT', C]",Homozygous,PTGES3,rs10579382,"initiatior_codon_variant', 'SO:0001589",Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No,No,initiatior codon variant,NaN
4,"['CTGTG', C]",Heterozygous,ELN,rs10579871,intron_variant,Conflicting_interpretations_of_pathogenicity,"Supravalvar_aortic_stenosis|Cutis_laxa', '_aut...","criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes,No,intron variant,2/10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Benign,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No,3 prime UTR variant,3/10
577,"['AT', A]",Homozygous,ATCAY,rs796133913,3_prime_UTR_variant,Uncertain_significance,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No,No,3 prime UTR variant,3/10
578,"['CT', C]",Heterozygous,FANCD2,rs797045572,intron_variant,Uncertain_significance,not_specified,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes,No,intron variant,2/10
579,"['C', CA]",Heterozygous,HSD17B3,rs8190483,intron_variant,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes,No,intron variant,2/10


In [85]:
merged_1.columns

Index(['allele', 'zygocity', 'Gene Name', 'rsid', 'consequence',
       'clinical_significance', 'associated_diseases', 'review_status',
       'origin', 'variant_type', 'variant_subtype', 'Phargkb_ann_exists',
       'is_mutation', 'Variant_is_precious', 'Gene_Match', 'Consequence',
       'Consequence_score'],
      dtype='object')

In [86]:
merged_1 = merged_1[['allele', 'zygocity', 'Gene Name', 'Gene_Match', 'rsid', 'consequence', 'Consequence_score', 'clinical_significance', 'associated_diseases', 'review_status',
       'origin', 'variant_type', 'variant_subtype', 'Phargkb_ann_exists',
       'is_mutation', 'Variant_is_precious']]
merged_1

,allele,zygocity,Gene Name,Gene_Match,rsid,consequence,Consequence_score,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious
0,"['AAGAGAG', A]",Heterozygous,TRPS1,No,rs10546472,5_prime_UTR_variant,3/10,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No
1,"['AAGAGAG', A]",Heterozygous,TRPS1,No,rs10546472,5_prime_UTR_variant,3/10,Uncertain_significance,Trichorhinophalangeal_syndrome,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
2,"['ACCTT', A]",Heterozygous,ADAMTS17,No,rs10549565,3_prime_UTR_variant,3/10,Benign,"Weill-Marchesani_4_syndrome', '_recessive|not_...","criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,No
3,"['CAT', C]",Homozygous,PTGES3,No,rs10579382,"initiatior_codon_variant', 'SO:0001589",NaN,Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No
4,"['CTGTG', C]",Heterozygous,ELN,No,rs10579871,intron_variant,2/10,Conflicting_interpretations_of_pathogenicity,"Supravalvar_aortic_stenosis|Cutis_laxa', '_aut...","criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,"['AT', A]",Homozygous,ATCAY,No,rs796133913,3_prime_UTR_variant,3/10,Benign,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
577,"['AT', A]",Homozygous,ATCAY,No,rs796133913,3_prime_UTR_variant,3/10,Uncertain_significance,Cayman_type_cerebellar_ataxia,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
578,"['CT', C]",Heterozygous,FANCD2,No,rs797045572,intron_variant,2/10,Uncertain_significance,not_specified,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,Yes
579,"['C', CA]",Heterozygous,HSD17B3,No,rs8190483,intron_variant,2/10,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes


In [87]:
null_rows = merged_1[merged_1['Consequence_score'].isnull()]
null_rows

,allele,zygocity,Gene Name,Gene_Match,rsid,consequence,Consequence_score,clinical_significance,associated_diseases,review_status,origin,variant_type,variant_subtype,Phargkb_ann_exists,is_mutation,Variant_is_precious
3,"['CAT', C]",Homozygous,PTGES3,No,rs10579382,"initiatior_codon_variant', 'SO:0001589",NaN,Benign,not_provided,no_assertion_criteria_provided,germline,indel,Deletion,MayBe No,No,No
33,"['G', GT]",Heterozygous,MYOZ2,No,rs112369914,NA,NaN,Benign,not_specified|not_provided,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes
36,"['A', AGGCCTTGCGTCGT]",Heterozygous,SPIRE2,No,rs11275235,NA,NaN,Conflicting_interpretations_of_pathogenicity,not_specified|not_provided,"criteria_provided', '_conflicting_interpretations",germline,indel,Insertion,MayBe No,No,Yes
110,"['C', CA]",Heterozygous,CDH3,No,rs11440411,NA,NaN,Benign,EEM_syndrome,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No
116,"['AG', A]",Homozygous,MYBPC3,No,rs11570050,NA,NaN,Benign/Likely_benign,Cardiomyopathy|Hypertrophic_cardiomyopathy|Lef...,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Deletion,MayBe No,No,Yes
118,"['C', CTG]",Heterozygous,ARSB,No,rs1255033691,NA,NaN,Benign,Mucopolysaccharidosis_type_6,"criteria_provided', '_single_submitter",germline,indel,Insertion,MayBe No,No,No
154,"['GCCCACCCCAC', G]",Heterozygous,VSX1,No,rs144366921,NA,NaN,Uncertain_significance,Polymorphous_corneal_dystrophy,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
155,"['GCCCACCCCAC', G]",Heterozygous,VSX1,No,rs144366921,NA,NaN,Benign,Polymorphous_corneal_dystrophy,"criteria_provided', '_single_submitter",germline,indel,Deletion,MayBe No,No,No
165,"['T', TCGGCG]",Heterozygous,ALG11,No,rs148013251,genic_upstream_transcript_variant,NaN,Benign/Likely_benign,Wilson_disease|ALG11-CDG|Congenital_disorder_o...,"criteria_provided', '_multiple_submitters', '_...",germline,indel,Insertion,MayBe No,No,Yes
171,"['G', GT]",Heterozygous,KMT2C,No,rs150073007,nonsense,NaN,Benign,Intellectual_disability|not_specified,"criteria_provided', '_single_submitter",NaN,indel,Insertion,MayBe No,No,Yes


In [88]:
merged_1.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_files_all/KHGLBS535/KHGLBS535_InDel.xlsx', index=False)

In [ ]:
import pandas as pd

# Load datasets
dataset1 = pd.read_csv('dataset1.csv')  # Replace with your file path
dataset2 = pd.read_csv('dataset2.csv')  # Replace with your file path

# Preprocess dataset1 to handle comma-separated gene names
dataset1['Gene Name'] = dataset1['Gene Name'].str.replace(',', ' ').str.split()

# Create a dictionary to map gene names to conditions from dataset1
gene_to_condition = {}
for _, row in dataset1.iterrows():
    for gene in row['Gene Name']:
        gene_to_condition[gene] = row['Condition']

# Populate the condition column in dataset2
dataset2['Condition'] = dataset2['Gene Name'].map(gene_to_condition)

# Save the updated dataset2 with the new Condition column
dataset2.to_csv('updated_dataset2.csv', index=False)  # Replace with your desired file name


In [1]:
import pandas as pd

In [3]:
data = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Cerebellarataxia_Lit_final_Positions_hg38_hg37.csv', sep = '\t')
data

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37,Chrom-pos-Ref-Alt_38,rsID_hg38,Chrom-pos-Ref-Alt_37,rsID_hg37
0,OTC,rs200016637,rs200016637,rs200016637,"nan, X-38352177-CTTC-C,",rs200016637,"nan, X-38211430-CTTC-C",rs200016637
1,OTC,rs201351170,rs201351170,rs201351170,"X-38352183-TCTAT-T,",rs201351170,X-38211436-TCTAT-T,rs201351170
2,OTC,rs57752938,rs57752938,rs57752938,"X-38352181-TTTCT-T,",rs57752938,X-38211434-TTTCT-T,rs57752938
3,OTC,rs5917572,rs5917572,rs5917572,"X-38352256-A-G, X-38352256-A-T,",rs5917572,"X-38211509-A-G, X-38211509-A-T",rs5917572
4,OTC,rs5963030,rs5963030,rs5963030,"X-38352332-G-A,",rs5963030,X-38211585-G-A,rs5963030
...,...,...,...,...,...,...,...,...
764,PDP1,c.575dupT,PDP1:c.575dupT,PDP1:c.575dupT,"8-93922634-G-GG, 8-93922559-T-TT, nan",NaN,"8-94934787-T-TT, 8-94934862-G-GG, nan",NaN
765,PDP1,c.277G>T,PDP1:c.277G>T,PDP1:c.277G>T,"8-93858083-G-T, 8-93922336-G-T, 8-93922261-G-T...",rs267606938,"8-94870311-G-T, 8-94934564-G-T, 8-94934489-G-T...",rs267606938
766,DLD,c.259C>T,DLD:c.259C>T,DLD:c.259C>T,"7-107902385-C-T, nan",rs925493461,"7-107542830-C-T, nan",rs925493461
767,DLD,c.803_804delAG,DLD:c.803_804delAG,DLD:c.803_804delAG,"7-107916869-CTT-C, 7-107915623-CAG-C, 7-107915...",rs764611160,"7-107556068-CAG-C, 7-107557314-CTT-C, 7-107556...",rs764611160


In [4]:
data.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Cerebellarataxia_Lit_final_Positions_hg38_hg37.xlsx', index=False)

In [12]:
vcf = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/vcf_4/KHBSTLGPTTL2/KHBSTLGPTTL2_final_depth.tsv', sep="\t", low_memory=False)
#vcf.columns = ['CHROM', 'POS', 'rsID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'SAMPLE']
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,1.949000e-03,16,0,8,0
1,chr1,14599,rs707680,T,A,.,PASS,"ADP=30;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.8524,0....",0/1,37,30,30,19,11,36.67%,1.594000e-04,19,0,11,0
2,chr1,14604,rs541940975,A,G,.,PASS,"ADP=31;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.8524,.,...",0/1,37,31,31,20,11,35.48%,1.665900e-04,20,0,11,0
3,chr1,14610,rs878986575,T,C,.,PASS,ADP=31;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,37,31,31,20,11,35.48%,1.665900e-04,20,0,11,0
4,chr1,14752,rs17855866,G,A,.,PASS,ADP=120;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX...,0/1,84,120,120,94,26,21.67%,3.263200e-09,69,25,24,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82332,chrY,57209613,.,A,G,.,PASS,ADP=21;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,62,21,21,6,15,71.43%,5.499400e-07,2,4,10,5
82333,chrY,57210759,.,C,G,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,77,58,58,35,22,37.93%,1.917900e-08,21,14,14,8
82334,chrY,57210867,.,T,C,.,PASS,ADP=25;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,23,25,25,17,7,28%,4.700600e-03,4,13,1,6
82335,chrY,57211440,.,A,G,.,PASS,ADP=19;WT=0;HET=1;HOM=0;NC=0;CSQ=G|intron_vari...,0/1,23,19,19,12,7,36.84%,3.992600e-03,11,1,6,1


In [13]:
vcf['HET'] = vcf['INFO'].str.extract(r'HET=(\d)')
vcf['HOM'] = vcf['INFO'].str.extract(r'HOM=(\d)')

# Create a new column 'Zygosity' based on conditions
vcf['Zygosity'] = ''

vcf.loc[vcf['HOM'] == '1', 'Zygosity'] = 'Homozygous'
vcf.loc[vcf['HET'] == '1', 'Zygosity'] = 'Heterozygous'
vcf['GT'] = vcf['GT'].astype(str)

vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,1.949000e-03,16,0,8,0,1,0,Heterozygous
1,chr1,14599,rs707680,T,A,.,PASS,"ADP=30;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.8524,0....",0/1,37,30,30,19,11,36.67%,1.594000e-04,19,0,11,0,1,0,Heterozygous
2,chr1,14604,rs541940975,A,G,.,PASS,"ADP=31;WT=0;HET=1;HOM=0;NC=0;ASP;CAF=0.8524,.,...",0/1,37,31,31,20,11,35.48%,1.665900e-04,20,0,11,0,1,0,Heterozygous
3,chr1,14610,rs878986575,T,C,.,PASS,ADP=31;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,37,31,31,20,11,35.48%,1.665900e-04,20,0,11,0,1,0,Heterozygous
4,chr1,14752,rs17855866,G,A,.,PASS,ADP=120;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX...,0/1,84,120,120,94,26,21.67%,3.263200e-09,69,25,24,2,1,0,Heterozygous
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82332,chrY,57209613,.,A,G,.,PASS,ADP=21;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,62,21,21,6,15,71.43%,5.499400e-07,2,4,10,5,1,0,Heterozygous
82333,chrY,57210759,.,C,G,.,PASS,ADP=58;WT=0;HET=1;HOM=0;NC=0;CSQ=G|non_coding_...,0/1,77,58,58,35,22,37.93%,1.917900e-08,21,14,14,8,1,0,Heterozygous
82334,chrY,57210867,.,T,C,.,PASS,ADP=25;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,23,25,25,17,7,28%,4.700600e-03,4,13,1,6,1,0,Heterozygous
82335,chrY,57211440,.,A,G,.,PASS,ADP=19;WT=0;HET=1;HOM=0;NC=0;CSQ=G|intron_vari...,0/1,23,19,19,12,7,36.84%,3.992600e-03,11,1,6,1,1,0,Heterozygous


In [14]:
vcf['CSQ'] = vcf['INFO'].str.extract(r'CSQ=(.*)')

vcf['csq'] = vcf['CSQ'].str.split(',')
vcf = vcf.explode('csq')
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,CSQ,csq
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82336,chrY,57211716,.,T,C,.,PASS,ADP=38;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,29,38,38,29,9,23.68%,0.001144,23,6,3,6,1,0,Heterozygous,C|intron_variant&non_coding_transcript_variant...,C|non_coding_transcript_exon_variant|MODIFIER|...
82336,chrY,57211716,.,T,C,.,PASS,ADP=38;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,29,38,38,29,9,23.68%,0.001144,23,6,3,6,1,0,Heterozygous,C|intron_variant&non_coding_transcript_variant...,C|intron_variant&non_coding_transcript_variant...
82336,chrY,57211716,.,T,C,.,PASS,ADP=38;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,29,38,38,29,9,23.68%,0.001144,23,6,3,6,1,0,Heterozygous,C|intron_variant&non_coding_transcript_variant...,C|downstream_gene_variant|MODIFIER|WASH6P|ENSG...
82336,chrY,57211716,.,T,C,.,PASS,ADP=38;WT=0;HET=1;HOM=0;NC=0;CSQ=C|intron_vari...,0/1,29,38,38,29,9,23.68%,0.001144,23,6,3,6,1,0,Heterozygous,C|intron_variant&non_coding_transcript_variant...,C|non_coding_transcript_exon_variant|MODIFIER|...


In [15]:
vcf[['Allele', 'Consequence', 'IMPACT', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'EXON', 'INTRON', 'HGVSc', 'HGVSp', 'cDNA_position', 'CDS_position', 'Protein_position', 'Amino_acids', 'Codons', 'Existing_variation', 'DISTANCE', 'STRAND', 'FLAGS', 'VARIANT_CLASS', 'SYMBOL_SOURCE', 'HGNC_ID', 'CANONICAL', 'MANE_SELECT', 'MANE_PLUS_CLINICAL', 'TSL', 'APPRIS', 'CCDS', 'ENSP', 'SWISSPROT', 'TREMBL', 'UNIPARC', 'UNIPROT_ISOFORM', 'SOURCE', 'GENE_PHENO', 'SIFT', 'PolyPhen', 'DOMAINS', 'miRNA', 'HGVS_OFFSET', 'AF', 'AFR_AF', 'AMR_AF', 'EAS_AF', 'EUR_AF', 'SAS_AF', 'gnomADe_AF', 'gnomADe_AFR_AF', 'gnomADe_AMR_AF', 'gnomADe_ASJ_AF', 'gnomADe_EAS_AF', 'gnomADe_FIN_AF', 'gnomADe_NFE_AF', 'gnomADe_OTH_AF', 'gnomADe_SAS_AF', 'gnomADg_AF', 'gnomADg_AFR_AF', 'gnomADg_AMI_AF', 'gnomADg_AMR_AF', 'gnomADg_ASJ_AF', 'gnomADg_EAS_AF', 'gnomADg_FIN_AF', 'gnomADg_MID_AF', 'gnomADg_NFE_AF', 'gnomADg_OTH_AF', 'gnomADg_SAS_AF', 'MAX_AF', 'MAX_AF_POPS', 'CLIN_SIG', 'SOMATIC', 'PHENO', 'PUBMED', 'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE', 'TRANSCRIPTION_FACTORS', 'ClinVar', 'ClinVar_CLNSIG', 'ClinVar_CLNREVSTAT', 'ClinVar_CLNDN']] = vcf['csq'].str.split('|', expand=True)
vcf

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,GT,GQ,SDP,DP,RD,AD,FREQ,PVAL,RDF,RDR,ADF,ADR,HET,HOM,Zygosity,CSQ,csq,Allele,Consequence,IMPACT,SYMBOL,Gene,Feature_type,Feature,BIOTYPE,EXON,INTRON,HGVSc,HGVSp,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,DISTANCE,STRAND,FLAGS,VARIANT_CLASS,SYMBOL_SOURCE,HGNC_ID,CANONICAL,MANE_SELECT,MANE_PLUS_CLINICAL,TSL,APPRIS,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,UNIPROT_ISOFORM,SOURCE,GENE_PHENO,SIFT,PolyPhen,DOMAINS,miRNA,HGVS_OFFSET,AF,AFR_AF,AMR_AF,EAS_AF,EUR_AF,SAS_AF,gnomADe_AF,gnomADe_AFR_AF,gnomADe_AMR_AF,gnomADe_ASJ_AF,gnomADe_EAS_AF,gnomADe_FIN_AF,gnomADe_NFE_AF,gnomADe_OTH_AF,gnomADe_SAS_AF,gnomADg_AF,gnomADg_AFR_AF,gnomADg_AMI_AF,gnomADg_AMR_AF,gnomADg_ASJ_AF,gnomADg_EAS_AF,gnomADg_FIN_AF,gnomADg_MID_AF,gnomADg_NFE_AF,gnomADg_OTH_AF,gnomADg_SAS_AF,MAX_AF,MAX_AF_POPS,CLIN_SIG,SOMATIC,PHENO,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,TRANSCRIPTION_FACTORS,ClinVar,ClinVar_CLNSIG,ClinVar_CLNREVSTAT,ClinVar_CLNDN
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A,downstream_gene_variant,MODIFIER,DDX11L1,ENSG00000223972,Transcript,ENST00000450305,transcribed_unprocessed_pseudogene,,,,,,,,,,rs707679,920,1,,SNV,HGNC,HGNC:37102,YES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.1311,0.09045,0.1524,0.1382,0.2095,0.06325,0.1167,0.05789,0.1594,0.1394,0.1409,0.2095,gnomADg_ASJ,,,,,,,,,,,,,
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|DDX11L2|ENS...,A,downstream_gene_variant,MODIFIER,DDX11L2,ENSG00000290825,Transcript,ENST00000456328,lncRNA,,,,,,,,,,rs707679,181,1,,SNV,EntrezGene,,YES,,,1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.1311,0.09045,0.1524,0.1382,0.2095,0.06325,0.1167,0.05789,0.1594,0.1394,0.1409,0.2095,gnomADg_ASJ,,,,,,,,,,,,,
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|intron_variant&non_coding_transcript_variant...,A,intron_variant&non_coding_transcript_variant,MODIFIER,WASH7P,ENSG00000227232,Transcript,ENST00000488147,unprocessed_pseudogene,,10/10,ENST00000488147.1:n.1254-89C>T,,,,,,,rs707679,,-1,,SNV,HGNC,HGNC:38034,YES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.1311,0.09045,0.1524,0.1382,0.2095,0.06325,0.1167,0.05789,0.1594,0.1394,0.1409,0.2095,gnomADg_ASJ,,,,,,,,,,,,,
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|downstream_gene_variant|MODIFIER|MIR6859-1|E...,A,downstream_gene_variant,MODIFIER,MIR6859-1,ENSG00000278267,Transcript,ENST00000619216,miRNA,,,,,,,,,,rs707679,2779,-1,,SNV,HGNC,HGNC:50039,YES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.1311,0.09045,0.1524,0.1382,0.2095,0.06325,0.1167,0.05789,0.1594,0.1394,0.1409,0.2095,gnomADg_ASJ,,,,,,,,,,,,,
0,chr1,14590,rs707679,G,A,.,PASS,ADP=24;WT=0;HET=1;HOM=0;NC=0;ASP;GENEINFO=DDX1...,0/1,27,24,24,16,8,33.33%,0.001949,16,0,8,0,1,0,Heterozygous,A|downstream_gene_variant|MODIFIER|DDX11L1|ENS...,A|regulatory_region_variant|MODIFIER|||Regulat...,A,regulatory_region_variant,MODIFIER,,,RegulatoryFeature,ENSR00001164745,enhancer,,,,,,,,,,rs707679,,,,SNV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0.1311,0.09045,0.1524,0.1382,0.2095,0.06325,0.1167,0.05789,0.1594,0.1394,0.1409,0.2095,gnomADg_ASJ,,,,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [17]:
df = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/Psoriasis_Lit_variants.csv', sep = '\t')
df

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37
0,TNIP1,rs17728338,rs17728338,rs17728338
1,TNFAIP3,rs610604,rs610604,rs610604
2,CARD14,p.Arg38Cys,CARD14:p.Arg38Cys,CARD14:p.Arg38Cys
3,CARD14,p.Arg62Gln,CARD14:p.Arg62Gln,CARD14:p.Arg62Gln
4,CARD14,p.Arg69Trp,CARD14:p.Arg69Trp,CARD14:p.Arg69Trp
5,CARD14,rs281875215,rs281875215,rs281875215
6,CARD14,p.Met119Val,CARD14:p.Met119Val,CARD14:p.Met119Val
7,CARD14,p.Leu124Pro,CARD14:p.Leu124Pro,CARD14:p.Leu124Pro
8,CARD14,p.Glu138Ala,CARD14:p.Glu138Ala,CARD14:p.Glu138Ala
9,CARD14,p.Glu138Lys,CARD14:p.Glu138Lys,CARD14:p.Glu138Lys


In [20]:
data = pd.read_csv(r'C:/Users/GenepoweRx_Madhu/Downloads/hg38_results_psoriasis.tsv', sep = '\t')
data = data.rename({'Uploaded_variant': 'Uploaded_variant_hg38'}, axis=1)
data

,Uploaded_variant_hg38,Chrom-pos-Ref-Alt,rsID
0,CARD14:p.Arg38Cys,"17-80181550-C-T,","rs281875217,"
1,CARD14:p.Arg62Gln,"17-80188471-AG-CA,",-
2,CARD14:p.Arg62Gln,"17-80181623-G-A,","rs115582620,"
3,CARD14:p.Arg69Trp,"17-80192583-AGA-TGG,",-
4,CARD14:p.Arg69Trp,"17-80181643-C-T,","rs375624435,"
...,...,...,...
57,rs281875216,"17-80184074-C-T,","rs281875216,"
58,rs144475004,"17-80184089-G-C,","rs144475004,"
59,rs587777763,"17-80182795-G-C,","rs587777763,"
60,rs587777763,"17-80182795-G-A,","rs587777763,"


In [28]:
merged = pd.merge(df, data, on = 'Uploaded_variant_hg38', how = 'left')
merged

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37,Chrom-pos-Ref-Alt,rsID
0,TNIP1,rs17728338,rs17728338,rs17728338,5-151098757-G-A,rs17728338
1,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-T,rs610604
2,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-A,rs610604
3,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-C,rs610604
4,CARD14,p.Arg38Cys,CARD14:p.Arg38Cys,CARD14:p.Arg38Cys,"17-80181550-C-T,","rs281875217,"
...,...,...,...,...,...,...
57,CARD14,rs281875216,rs281875216,rs281875216,"17-80184074-C-T,","rs281875216,"
58,CARD14,rs144475004,rs144475004,rs144475004,"17-80184089-G-C,","rs144475004,"
59,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-C,","rs587777763,"
60,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-A,","rs587777763,"


In [29]:
merged['chrom'] = merged['Chrom-pos-Ref-Alt'].str.split(',')
merged = merged.explode('chrom')
merged

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37,Chrom-pos-Ref-Alt,rsID,chrom
0,TNIP1,rs17728338,rs17728338,rs17728338,5-151098757-G-A,rs17728338,5-151098757-G-A
1,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-T,rs610604,6-137878280-G-T
2,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-A,rs610604,6-137878280-G-A
3,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-C,rs610604,6-137878280-G-C
4,CARD14,p.Arg38Cys,CARD14:p.Arg38Cys,CARD14:p.Arg38Cys,"17-80181550-C-T,","rs281875217,",17-80181550-C-T
...,...,...,...,...,...,...,...
59,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-C,","rs587777763,",
60,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-A,","rs587777763,",17-80182795-G-A
60,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-A,","rs587777763,",
61,CARD14,rs886041402,rs886041402,rs886041402,"17-80182791-G-A,","rs886041402,",17-80182791-G-A


In [30]:
merged['POS'] = merged['chrom'].str.split('-').str[1]
merged

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37,Chrom-pos-Ref-Alt,rsID,chrom,POS
0,TNIP1,rs17728338,rs17728338,rs17728338,5-151098757-G-A,rs17728338,5-151098757-G-A,151098757
1,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-T,rs610604,6-137878280-G-T,137878280
2,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-A,rs610604,6-137878280-G-A,137878280
3,TNFAIP3,rs610604,rs610604,rs610604,6-137878280-G-C,rs610604,6-137878280-G-C,137878280
4,CARD14,p.Arg38Cys,CARD14:p.Arg38Cys,CARD14:p.Arg38Cys,"17-80181550-C-T,","rs281875217,",17-80181550-C-T,80181550
...,...,...,...,...,...,...,...,...
59,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-C,","rs587777763,",,NaN
60,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-A,","rs587777763,",17-80182795-G-A,80182795
60,CARD14,rs587777763,rs587777763,rs587777763,"17-80182795-G-A,","rs587777763,",,NaN
61,CARD14,rs886041402,rs886041402,rs886041402,"17-80182791-G-A,","rs886041402,",17-80182791-G-A,80182791


In [24]:
merged.isnull().sum()

Gene                     0
Change                   0
Uploaded_variant_hg38    0
Uploaded_variant_hg37    0
Chrom-pos-Ref-Alt        2
rsID                     4
dtype: int64

In [31]:
merged.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/psy.xlsx', index=False)

# POS and CHROM extraction from the condition final sheet

In [42]:
import pandas as pd
x = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Epilepsy_Lit_final_Positions_hg38_hg37 (1).xlsx')
x['chrom'] = x['Chrom-pos-Ref-Alt_38'].str.split(',')
x = x.explode('chrom')

x['CHROM'] = x['chrom'].str.split('-').str[0]

# Function to add 'chr' prefix conditionally
def add_chr_prefix(chromosome):
    if pd.notnull(chromosome) and chromosome.strip() != '':
        return 'chr' + str(chromosome)
    else:
        return chromosome

# Applying the function to the 'chromosome' column
x['CHROM'] = x['CHROM'].apply(add_chr_prefix)
x['CHROM'] = x['CHROM'].str.strip()
x['CHROM'] = x['CHROM'].str.replace(r'\s+', '')
x['POS'] = x['chrom'].str.split('-').str[1]

x.dropna(subset=['CHROM'], inplace=True)
# Drop rows with empty cells after removing leading and trailing whitespaces
x['CHROM'] = x['CHROM'].str.strip()
x['POS'] = x['POS'].str.strip()
# Dropping rows with empty cells and NaN values in both 'chromosome' and 'position' columns
x.dropna(subset=['CHROM', 'POS'], inplace=True)
x = x[['CHROM', 'POS']]
x['Literature'] = 'Yes'
x.drop_duplicates(subset='POS', inplace=True)
x

,CHROM,POS,Literature
0,chrX,100407579,Yes
2,chrX,100407852,Yes
3,chrX,100406643,Yes
6,chr2,166046970,Yes
7,chr2,165998075,Yes
...,...,...,...
299,chr6,145627532,Yes
299,chr6,145640407,Yes
300,chr6,145635288,Yes
300,chr6,145627510,Yes


In [41]:
x.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Condition_pos/Epilepsy_pos_new.xlsx', index=False)

# ---------------------------------------------------------------------------------------------------------------

In [2]:
y = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Migranes_Lit_final_Positions_hg38_hg37.xlsx')
y

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37,Chrom-pos-Ref-Alt_38,rsID_hg38,Chrom-pos-Ref-Alt_37,rsID_hg37
0,COMT,rs4680,rs4680,rs4680,"22-19963748-G-A,",rs4680,22-19951271-G-A,rs4680
1,CACNA1A,rs10405121,rs10405121,rs10405121,"19-13228314-G-A, 19-13228314-G-C,",rs10405121,"19-13339128-G-C, 19-13339128-G-A,",rs10405121
2,CACNA1A,c.653C > T,CACNA1A:c.653C>T,CACNA1A:c.653C>T,"19-13365448-G-A, 19-13214557-G-A, 19-13230151-...","rs121908225, rs1483005213, rs2056673232","19-13476262-G-A, 19-13366069-G-A,",rs121908225
3,CACNA1A,c.1088T > C,CACNA1A:c.1088T>C,CACNA1A:c.1088T>C,"19-13334488-A-G, 19-13214246-A-G, 19-13359691-...","rs959681605, rs1291528882","19-13445302-A-G, 19-13346465-A-G,",NaN
4,CACNA1A,c.4037G > A,CACNA1A:c.4037G>A,CACNA1A:c.4037G>A,"19-13262789-C-T,",rs121908230,"19-13373603-C-T,",rs121908230
...,...,...,...,...,...,...,...,...
62,KCNK18,p.Phe139Trp,KCNK18:p.Phe139Trp,KCNK18:p.Phe139Trp,NaN,NaN,NaN,NaN
63,PRRT2,c.649dupC,PRRT2:c.649dupC,PRRT2:c.649dupC,"16-29814394-A-AA, 16-29813703-C-CC, 16-2981481...",NaN,16-29825024-C-CC,NaN
64,PRRT2,c.649delC,PRRT2:c.649delC,PRRT2:c.649delC,"16-29813702-CC-C, 16-29814393-AA-A, 16-2981481...",NaN,16-29825023-CC-C,NaN
65,PRRT2,c.843C>G,PRRT2:c.843C>G,PRRT2:c.843C>G,16-29815009-C-G,NaN,NaN,NaN


In [5]:
y['CHROM'] = y['Chrom-pos-Ref-Alt_38'].str.split('-').str[1]
y

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37,Chrom-pos-Ref-Alt_38,rsID_hg38,Chrom-pos-Ref-Alt_37,rsID_hg37,CHROM
0,COMT,rs4680,rs4680,rs4680,"22-19963748-G-A,",rs4680,22-19951271-G-A,rs4680,19963748
1,CACNA1A,rs10405121,rs10405121,rs10405121,"19-13228314-G-A, 19-13228314-G-C,",rs10405121,"19-13339128-G-C, 19-13339128-G-A,",rs10405121,13228314
2,CACNA1A,c.653C > T,CACNA1A:c.653C>T,CACNA1A:c.653C>T,"19-13365448-G-A, 19-13214557-G-A, 19-13230151-...","rs121908225, rs1483005213, rs2056673232","19-13476262-G-A, 19-13366069-G-A,",rs121908225,13365448
3,CACNA1A,c.1088T > C,CACNA1A:c.1088T>C,CACNA1A:c.1088T>C,"19-13334488-A-G, 19-13214246-A-G, 19-13359691-...","rs959681605, rs1291528882","19-13445302-A-G, 19-13346465-A-G,",NaN,13334488
4,CACNA1A,c.4037G > A,CACNA1A:c.4037G>A,CACNA1A:c.4037G>A,"19-13262789-C-T,",rs121908230,"19-13373603-C-T,",rs121908230,13262789
...,...,...,...,...,...,...,...,...,...
62,KCNK18,p.Phe139Trp,KCNK18:p.Phe139Trp,KCNK18:p.Phe139Trp,NaN,NaN,NaN,NaN,NaN
63,PRRT2,c.649dupC,PRRT2:c.649dupC,PRRT2:c.649dupC,"16-29814394-A-AA, 16-29813703-C-CC, 16-2981481...",NaN,16-29825024-C-CC,NaN,29814394
64,PRRT2,c.649delC,PRRT2:c.649delC,PRRT2:c.649delC,"16-29813702-CC-C, 16-29814393-AA-A, 16-2981481...",NaN,16-29825023-CC-C,NaN,29813702
65,PRRT2,c.843C>G,PRRT2:c.843C>G,PRRT2:c.843C>G,16-29815009-C-G,NaN,NaN,NaN,29815009


In [92]:
y['POS'] = y['Chr-Position-Ref-Alt'].str.split('-').str[1]


In [93]:
y

,Gene,Amino acid change,Uploaded_variant,Chr-Position-Ref-Alt,rsID,Variant mechanism,Gene Info,Publication,CHROM,POS
0,CFTR,p.G551D,CFTR:p.Gly551Asp,7-117587806-G-A,rs75527207,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117587806
1,CFTR,p.N1303K,CFTR:p.Asn1303Lys,7-117652877-C-G,rs80034486,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117652877
2,CFTR,p.R117H,CFTR:p.Arg117His,7-117710365-GA-AT,rs78655421,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117710365
3,CFTR,p.G85E,CFTR:p.Gly85Glu,7-117509123-G-A,rs75961395,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117509123
4,CFTR,p.R334W,CFTR:p.Arg334Trp,7-117540230-C-T,rs121909011,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117540230
...,...,...,...,...,...,...,...,...,...,...
209,TRPV6,p.A626P,TRPV6:p.Ala626Pro,7-142873480-C-G,-,NaN,Transient receptor potential vanilloid subfami...,PMID: 33610740,7,142873480
210,CLDN2,-,-,-,-,NaN,CLDN2 gene product belongs to the claudin prot...,NaN,,
211,CEL,-,-,-,-,NaN,"The enzyme carboxyl ester lipase (CEL), hydrol...",NaN,,
212,CASR,-,-,-,-,NaN,The CASR allows regulation of parathyroid horm...,NaN,,


In [94]:
y['REF_ALT'] = y['Chr-Position-Ref-Alt'].str.split('-').str[2:5].str.join('-')
y

,Gene,Amino acid change,Uploaded_variant,Chr-Position-Ref-Alt,rsID,Variant mechanism,Gene Info,Publication,CHROM,POS,REF_ALT
0,CFTR,p.G551D,CFTR:p.Gly551Asp,7-117587806-G-A,rs75527207,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117587806,G-A
1,CFTR,p.N1303K,CFTR:p.Asn1303Lys,7-117652877-C-G,rs80034486,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117652877,C-G
2,CFTR,p.R117H,CFTR:p.Arg117His,7-117710365-GA-AT,rs78655421,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117710365,GA-AT
3,CFTR,p.G85E,CFTR:p.Gly85Glu,7-117509123-G-A,rs75961395,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117509123,G-A
4,CFTR,p.R334W,CFTR:p.Arg334Trp,7-117540230-C-T,rs121909011,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386",7,117540230,C-T
...,...,...,...,...,...,...,...,...,...,...,...
209,TRPV6,p.A626P,TRPV6:p.Ala626Pro,7-142873480-C-G,-,NaN,Transient receptor potential vanilloid subfami...,PMID: 33610740,7,142873480,C-G
210,CLDN2,-,-,-,-,NaN,CLDN2 gene product belongs to the claudin prot...,NaN,,,
211,CEL,-,-,-,-,NaN,"The enzyme carboxyl ester lipase (CEL), hydrol...",NaN,,,
212,CASR,-,-,-,-,NaN,The CASR allows regulation of parathyroid horm...,NaN,,,


In [95]:
y.columns

Index(['Gene', 'Amino acid change', 'Uploaded_variant', 'Chr-Position-Ref-Alt',
       'rsID', 'Variant mechanism', 'Gene Info', 'Publication', 'CHROM', 'POS',
       'REF_ALT'],
      dtype='object')

In [96]:
y = y[['Gene', 'Amino acid change', 'Uploaded_variant', 'Chr-Position-Ref-Alt', 'CHROM', 'POS', 'REF_ALT', 'rsID', 
      'Variant mechanism', 'Gene Info', 'Publication']]
y

,Gene,Amino acid change,Uploaded_variant,Chr-Position-Ref-Alt,CHROM,POS,REF_ALT,rsID,Variant mechanism,Gene Info,Publication
0,CFTR,p.G551D,CFTR:p.Gly551Asp,7-117587806-G-A,7,117587806,G-A,rs75527207,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386"
1,CFTR,p.N1303K,CFTR:p.Asn1303Lys,7-117652877-C-G,7,117652877,C-G,rs80034486,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386"
2,CFTR,p.R117H,CFTR:p.Arg117His,7-117710365-GA-AT,7,117710365,GA-AT,rs78655421,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386"
3,CFTR,p.G85E,CFTR:p.Gly85Glu,7-117509123-G-A,7,117509123,G-A,rs75961395,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386"
4,CFTR,p.R334W,CFTR:p.Arg334Trp,7-117540230-C-T,7,117540230,C-T,rs121909011,NaN,Cystic fibrosis transmembrane conductance regu...,"PMID: 19092437, PMID: 34557648, PMID: 32153386"
...,...,...,...,...,...,...,...,...,...,...,...
209,TRPV6,p.A626P,TRPV6:p.Ala626Pro,7-142873480-C-G,7,142873480,C-G,-,NaN,Transient receptor potential vanilloid subfami...,PMID: 33610740
210,CLDN2,-,-,-,,,,-,NaN,CLDN2 gene product belongs to the claudin prot...,NaN
211,CEL,-,-,-,,,,-,NaN,"The enzyme carboxyl ester lipase (CEL), hydrol...",NaN
212,CASR,-,-,-,,,,-,NaN,The CASR allows regulation of parathyroid horm...,NaN


In [97]:
y.to_excel(r'C:/Users/GenepoweRx_Madhu/Desktop/Final_Pancreatitis_data.xlsx', index=False)

In [1]:
import pandas as pd
x = pd.read_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/Hypoglycemia_Lit_final_Positions_hg38_hg37.xlsx')
x['chrom'] = x['Chrom-pos-Ref-Alt_38'].str.split(',')
x = x.explode('chrom')

x['CHROM'] = x['chrom'].str.split('-').str[1]
x

,Gene,Change,Uploaded_variant_hg38,Uploaded_variant_hg37,Chrom-pos-Ref-Alt_38,rsID_hg38,Chrom-pos-Ref-Alt_37,rsID_hg37,chrom,CHROM
0,KCNJ11,c.78C>A,KCNJ11:c.78C>A,KCNJ11:c.78C>A,"11-17387753-G-T, 11-17388014-G-T, 11-17387786-G-T",NaN,"11-17409561-G-T, 11-17409300-G-T, 11-17409578-G-T",rs1393796559,11-17387753-G-T,17387753
0,KCNJ11,c.78C>A,KCNJ11:c.78C>A,KCNJ11:c.78C>A,"11-17387753-G-T, 11-17388014-G-T, 11-17387786-G-T",NaN,"11-17409561-G-T, 11-17409300-G-T, 11-17409578-G-T",rs1393796559,11-17388014-G-T,17388014
0,KCNJ11,c.78C>A,KCNJ11:c.78C>A,KCNJ11:c.78C>A,"11-17387753-G-T, 11-17388014-G-T, 11-17387786-G-T",NaN,"11-17409561-G-T, 11-17409300-G-T, 11-17409578-G-T",rs1393796559,11-17387786-G-T,17387786
1,KCNJ11,c.79C>T,KCNJ11:c.79C>T,KCNJ11:c.79C>T,11-17388013-G-A,rs752507753,"11-17409560-G-A, 11-17409577-G-A",rs752507753,11-17388013-G-A,17388013
2,KCNJ11,c.112A>G,KCNJ11:c.112A>G,KCNJ11:c.112A>G,"11-17387719-T-C, 11-17387980-T-C, 11-17386639-...",rs914289214,"11-17409527-T-C, 11-17409266-T-C, 11-17408186-T-C",rs914289214,11-17387719-T-C,17387719
...,...,...,...,...,...,...,...,...,...,...
189,ALDOB,c.841_842delAC,ALDOB:c.841_842delAC,ALDOB:c.841_842delAC,"9-101424999-AGT-A, 9-101429312-CAC-C, 9-101429...",NaN,9-104187281-AGT-A,NaN,9-101429406-GCC-G,101429406
189,ALDOB,c.841_842delAC,ALDOB:c.841_842delAC,ALDOB:c.841_842delAC,"9-101424999-AGT-A, 9-101429312-CAC-C, 9-101429...",NaN,9-104187281-AGT-A,NaN,,NaN
190,ALDOB,c.851T>C,ALDOB:c.851T>C,ALDOB:c.851T>C,NaN,NaN,NaN,NaN,NaN,NaN
191,ALDOB,c.1005C>G,ALDOB:c.1005C>G,ALDOB:c.1005C>G,"9-101421899-G-C,",rs78340951,9-104184181-G-C,rs78340951,9-101421899-G-C,101421899


In [2]:
x.to_excel(r'C:/Users/GenepoweRx_Madhu/Downloads/hypogly.xlsx', index=False)